In [1]:
from google.colab import files
uploaded = files.upload()

Saving monthly-car-sales.csv to monthly-car-sales.csv


In [2]:
# train_model.py

import pandas as pd
from xgboost import XGBRegressor

# ----------------------------------------------------------

df = pd.read_csv("monthly-car-sales.csv")

print(df.head())

# Convert Month to datetime (optional but helpful)
df["Month"] = pd.to_datetime(df["Month"])

# Sort by date just in case
df = df.sort_values("Month").reset_index(drop=True)


# ----------------------------------------------------------

for i in range(1, 7):
    df[f"lag_{i}"] = df["Sales"].shift(i)

# Remove rows without full lag history
df = df.dropna().reset_index(drop=True)
print("Dataset with lag features:")
print(df.head())


# ----------------------------------------------------------

feature_cols = [f"lag_{i}" for i in range(1, 7)]

X = df[feature_cols]
y = df["Sales"]

# Simple chronological train/test split
train_size = int(len(df) * 0.8)

X_train = X.iloc[:train_size]
y_train = y.iloc[:train_size]

X_test = X.iloc[train_size:]
y_test = y.iloc[train_size:]

print("Training samples:", len(X_train))
print("Testing samples:", len(X_test))


# ----------------------------------------------------------

model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(X_train, y_train)

print("Model training complete!")


# ----------------------------------------------------------

import os
os.makedirs("model", exist_ok=True)

model.get_booster().save_model("model/model.json")

print("Model saved successfully → model/model.json")


     Month  Sales
0  1960-01   6550
1  1960-02   8728
2  1960-03  12026
3  1960-04  14395
4  1960-05  14587
Dataset with lag features:
       Month  Sales    lag_1    lag_2    lag_3    lag_4    lag_5    lag_6
0 1960-07-01   9498  13791.0  14587.0  14395.0  12026.0   8728.0   6550.0
1 1960-08-01   8251   9498.0  13791.0  14587.0  14395.0  12026.0   8728.0
2 1960-09-01   7049   8251.0   9498.0  13791.0  14587.0  14395.0  12026.0
3 1960-10-01   9545   7049.0   8251.0   9498.0  13791.0  14587.0  14395.0
4 1960-11-01   9364   9545.0   7049.0   8251.0   9498.0  13791.0  14587.0
Training samples: 81
Testing samples: 21
Model training complete!
Model saved successfully → model/model.json


In [3]:
# app.py

from fastapi import FastAPI, Form
from fastapi.responses import HTMLResponse
import numpy as np
from xgboost import Booster

app = FastAPI()

# ----------------------------------------------------------
# 1. LOAD THE TRAINED XGBOOST MODEL (JSON)
# ----------------------------------------------------------
model = Booster()
model.load_model("model/model.json")


# ----------------------------------------------------------
# 2. HOME ROUTE (SHOW BASIC MESSAGE)
# ----------------------------------------------------------
@app.get("/", response_class=HTMLResponse)
def home():
    return """
    <h1>Car Sales Prediction API</h1>
    <p>Use POST /predict with 6 months of sales.</p>
    """


# ----------------------------------------------------------
# 3. PREDICTION ENDPOINT
#    Receives: 6 numbers from the HTML form
# ----------------------------------------------------------
@app.post("/predict")
def predict(
    m1: float = Form(...),
    m2: float = Form(...),
    m3: float = Form(...),
    m4: float = Form(...),
    m5: float = Form(...),
    m6: float = Form(...)
):
    # Convert inputs to correct shape (1,6)
    data = np.array([[m1, m2, m3, m4, m5, m6]])

    # Convert to XGBoost's DMatrix format
    from xgboost import DMatrix
    dmatrix_data = DMatrix(data)

    # Run prediction
    prediction = model.predict(dmatrix_data)[0]

    return {
        "prediction": float(prediction)
    }
